<div style="text-align: center; background-color: skyblue; font-family: 'Times New Roman', serif; color: white; padding: 20px; font-size: 36px; font-weight: bold; border-radius: 15px; box-shadow: 0px 6px 15px rgba(0, 0, 0, 0.2); transition: all 0.3s ease;">
    INTRODUCTION TO DATA SCIENCE<br>
    @ FIT-HCMUS, VNU-HCM<br>
    FINAL PROJECT 🏫
</div>
<style>
    div:hover {
        transform: scale(1.05);
        box-shadow: 0px 10px 20px rgba(0, 0, 0, 0.3);
    }
</style>

<div style="text-align: center; background-color: #ADD8E6; font-family: 'Times New Roman', serif; color: white; padding: 20px; font-size: 30px; font-weight: bold; border-radius: 10px; box-shadow: 0px 6px 15px rgba(0, 0, 0, 0.2); transition: all 0.3s ease;">
    THE ANALYSIS OF THE MOBILE PHONE MARKET AT MOBILE CITY STORE
</div>
<style>
    div:hover {
        transform: scale(1.05);
        box-shadow: 0px 10px 20px rgba(0, 0, 0, 0.3);
    }
</style>


---

<div style="text-align: center; background-color: #ADD8E6; font-family: 'Times New Roman', serif; color: white; padding: 25px; font-size: 20px; font-weight: bold; border-radius: 10px; box-shadow: 0px 6px 15px rgba(0, 0, 0, 0.2); transition: all 0.3s ease;">
    DATA MODELING
</div>
<style>
    div:hover {
        opacity: 0.8;
    }
</style>

**Người thực hiện**: Trương Tiến Anh

**MSSV**: 22120017


---

## Model: Random Forest Regressor

### Reason for Selection:
Do mục đích bài toán về việc dữ đoán giá điện thoại nên việc chọn model `Random Forest Regressor` là hợp lý. Bởi vì đầu ra của bài toán là giá trị số thực liên tục và model này được thiết kế để dự đoán các giá trị liên tục như vậy

### Import:

In [147]:
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

### Read data from file CSV:

In [148]:
data = pd.read_csv("../../data/processed_data.csv")

### Data Preprocessing:

Loại bỏ một vài cột không cần thiết cho dự đoán

In [149]:
cols_drop = ['Unnamed: 0', 'ten', 'duong_dan', 'danh_gia', 
             'so_luong_binh_luan', 'thiet_ke', 'do_sang_man_hinh', 
             'do_phan_giai_cam_sau', 'do_phan_giai_cam_truoc']

data.drop(columns=cols_drop, inplace=True)
print(data.columns)

Index(['loai_dien_thoai', 'thoi_gian_bao_hanh', 'mau_sac', 'gia_moi', 'gia_cu',
       'he_dieu_hanh', 'cpu', 'ram', 'bo_nho_trong', 'dung_luong_pin',
       'hang_dien_thoai', 'la_dien_thoai_cu', 'kich_thuoc_man_hinh',
       'tan_so_quet', 'loai_man_hinh', 'so_the_sim', 'loai_pin',
       'cong_suat_sac'],
      dtype='object')


Đối với các cột dạng Numeric ta sẽ thay các giá trị Nan bằng các giá trị phù hợp như: median, mean ...vvv

In [150]:
numeric_col = data.select_dtypes(include=['number']).columns

for column in numeric_col:
    data[column].fillna(data[column].mean(), inplace=True)

Đối với các cột dạng Non-numeric ta sẽ thay các giá trị Nan bằng giá trị xuất hiện nhiều nhất trong cột đó

In [151]:
object_col = data.select_dtypes(exclude=['number']).columns

for col in object_col:
    data[col].fillna(data[col].mode()[0], inplace=True)

In [152]:
print(data.isnull().sum())

loai_dien_thoai        0
thoi_gian_bao_hanh     0
mau_sac                0
gia_moi                0
gia_cu                 0
he_dieu_hanh           0
cpu                    0
ram                    0
bo_nho_trong           0
dung_luong_pin         0
hang_dien_thoai        0
la_dien_thoai_cu       0
kich_thuoc_man_hinh    0
tan_so_quet            0
loai_man_hinh          0
so_the_sim             0
loai_pin               0
cong_suat_sac          0
dtype: int64


In [153]:
print(data.dtypes)

loai_dien_thoai         object
thoi_gian_bao_hanh     float64
mau_sac                 object
gia_moi                float64
gia_cu                 float64
he_dieu_hanh            object
cpu                     object
ram                    float64
bo_nho_trong           float64
dung_luong_pin         float64
hang_dien_thoai         object
la_dien_thoai_cu          bool
kich_thuoc_man_hinh    float64
tan_so_quet            float64
loai_man_hinh           object
so_the_sim               int64
loai_pin                object
cong_suat_sac          float64
dtype: object


Chuyển các cột object thành số bằng One-Hot Encoding

In [154]:
for col in object_col:
    # Tạo DataFrame mới với các cột dummy
    dummies = pd.get_dummies(data[col], prefix=col, drop_first=True)
    
    # Xóa cột ban đầu và thêm các cột dummy vào DataFrame
    data = pd.concat([data, dummies], axis=1)
    data.drop(col, axis=1, inplace=True)

Chuẩn bị dữ liệu và chia dữ liệu train và test
- features: các biến đầu vào để tiến hành phân tính và học 
- target: các biến đầu ra của kết quả dự đoán

In [155]:
features = [col for col in data.columns if col != 'gia_moi'] #Lấy ra các cột trong data ngoại trừa 'gia_moi'

target = 'gia_moi'

X = data[features] 
y = data[target]

# Chia dữ liệu train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Model Training:

Ta sẽ huấn luyện mô hình Random Forest

In [156]:
model = RandomForestRegressor()

start_time = time.time()
model.fit(X_train, y_train)
end_time = time.time()

### Model Evaluation:

Random Forest Regressor là một mô hình hồi quy để dự đoán các giá trị liên tục. Các chỉ số đánh giá quan trọng gồm:

**Mean Absolute Error (MAE):** dùng để đo lường độ sai lệch trung bình tuyệt đối giữa giá trị dự đoán và giá trị thực tế.
$$
\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} \left| y_i - \hat{y}_i \right|
$$

**Mean Squared Error (MSE)**: dùng để đo lường độ sai lệch bình phương trung bình giữa giá trị dự đoán và giá trị thực tế
$$
\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} \left( y_i - \hat{y}_i \right)^2
$$


**R² Score (Coefficient of Determination)**: dùng để đo lường mức độ mà mô hình giải thích được sự biến đổi của dữ liệu. R² có giá trị từ 0 đến 1 (càng gần 1, mô hình càng tốt).
$$
R^2 = 1 - \frac{\sum_{i=1}^{n} \left( y_i - \hat{y}_i \right)^2}{\sum_{i=1}^{n} \left( y_i - \bar{y} \right)^2}
$$

In [157]:
y_predict = model.predict(X_test)

mae = mean_absolute_error(y_test, y_predict)
mse = mean_squared_error(y_test, y_predict)
r2 = r2_score(y_test, y_predict)

print("\nĐánh giá mô hình Random Forest Regressor:")
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"R2 Score: {r2}")

training_time = end_time - start_time
print(f"Thời gian huấn luyện mô hình: {training_time:.2f} giây")


Đánh giá mô hình Random Forest Regressor:
MAE: 367335.51100407366
MSE: 825734724693.8856
R2 Score: 0.9826380857528605
Thời gian huấn luyện mô hình: 59.66 giây


Nhận xét kết quả trên:
- R2 Score là khoảng 0.98 được coi là 1 mô hình tốt, cho thấy mô hình có khả năng dự đoán chính xác với phần lớn dữ liệu.
- MSE vẫn còn lớn khoảng > 800 tỷ, cho thấy sai số của mô hình không đồng đều và có thể có một số dự đoán lệch xa thực tế.
